# Goal
Investigate if there is anything interesting in the interaction of pipeline steps and/or individual pipeline components and the rmse.

Obviously very adhoc but such is the nature of exploratory research

In [2]:
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from collections import defaultdict
from plotly.subplots import make_subplots
from scipy import stats
from scipy.stats import friedmanchisquare
import numpy as np
from itertools import chain

import json

from constants import SEED
from datasets import load_dataset

def get_all_results_paths(root_dir: Path):
    return list(root_dir.glob("**/*results.json"))

def single_layer_unnest(node: dict, keys: list[str]):
    """
    Unnest a single layer of a nested dictionary
    """
    to_merge_later = {}
    to_delete_later = []
    for top_key, top_item in node.items():
        if isinstance(top_item, dict):
            for child_key, child_item in top_item.items():
                if child_key in keys:
                    to_merge_later[f"{top_key}_{child_key}"] = child_item
                    to_delete_later.append((top_key, child_key))
    for top_key, child_key in to_delete_later:
        del node[top_key][child_key]
    for new_key, new_item in to_merge_later.items():
        node[new_key] = new_item
    return node

def get_all_results(root_dir: Path, response_is_normalised: bool = False, log_response: bool = False):
    results = []
    for path in get_all_results_paths(root_dir):
        result = json.load(open(path))
        ## Handle nested metrics
        result = single_layer_unnest(result, ["train", "val", "test"])
        to_delete = []
        for key, val in result.items():
            if not val:
                to_delete.append(key)
        for key in to_delete:
            del result[key]
        ## Map model names
        result["model"] = result["model"].replace("relu_mlp", "mlp")
        result["model"] = result["model"].replace("no_activation_mlp", "deep_linear")
        ## Handle finetune
        if "finetune" not in result:
            result["finetune"] = False
        ## Handle empty preprocessers
        if "preprocessers" not in result:
            result["preprocessers"] = ["empty"]
        ## Handle missing seed
        if "seed" not in result:
            seed_part = [x for x in path.parts if "seed" in x][0]
            result["seed"] = int(seed_part.split("_")[-1])
        ## Order preprocessers
        result["preprocessers"] = sorted(result["preprocessers"])

        nice_table_map = {
            "empty": "empty",
            "lemmatize": "Le",
            "lowercase": "Lo",
            "no_numbers_simple": "NN",
            "no_numbers_spacy": "NNSC",
            "no_punctuation": "NP",
            "no_stopwords_nltk": "NSN",
            "no_stopwords_spacy": "NSCC",
            "stem": "S",
        }
        cleaned_processers = [nice_table_map.get(x, x) for x in result["preprocessers"]]
        result["preprocesser(s)"] = ",".join(sorted(cleaned_processers))

        if response_is_normalised:
            normaliser_path = path.parent / "normalisation_params.json"
            with open(normaliser_path, "r") as f:
                normaliser_constants = json.load(f)
            std = normaliser_constants["std"]
            for col in result.keys():
                if "rmse" in col:
                    result[col] = result[col] * std
        if log_response:
            for col in result.keys():
                if "rmse" in col:
                    result[col] = np.log(result[col])
        results.append(result)
    return results


def permutation_test(x, y, n_permutations=100000, one_sided: bool = False):
    """
    Perform permutation test for difference in means
    H0: mean(x) = mean(y)
    H1: mean(x) != mean(y) 

    If one_sided is True, then we test if mean(x) > mean(y)

    Return p-value and observed difference
    """
    observed_diff = np.mean(x) - np.mean(y)
    
    # Combine samples
    combined = np.concatenate([x, y])
    n_x = len(x)
    
    # Generate permutations
    diffs = np.zeros(n_permutations)
    for i in range(n_permutations):
        np.random.shuffle(combined)
        perm_x = combined[:n_x]
        perm_y = combined[n_x:]
        diffs[i] = np.mean(perm_x) - np.mean(perm_y)
    
    # Calculate one-sided p-value
    if one_sided:
        delta = diffs >= observed_diff
    else:
        delta = np.abs(diffs) >= np.abs(observed_diff)
    delta = np.concatenate([delta, [True]]) # Effectively do (C+1/N+1) as in https://arxiv.org/pdf/1603.05766
    p_value = np.mean(delta)
    
    return p_value, observed_diff

def process_latex_table(latex_table: str):
    latex_table = latex_table.replace("_", "-")
    latex_table = latex_table.replace("%", "\%")
    latex_table = latex_table.replace("rmse-test", "RMSE")
    latex_table = latex_table.replace("featuriser", "featurizer")
    latex_table = latex_table.replace("tokeniser", "tokenizer")
    latex_table = latex_table.replace("rmse-test", "rmse")
    latex_table = latex_table.replace("<", "\\textless")
    latex_table = latex_table.replace(">", "\\textgreater")
    latex_table = latex_table.replace("obl", "\\named{obl}")
    latex_table = latex_table.replace("jcp", "\\named{jcp}")
    return latex_table

def simple_format(x: float):
    if abs(x) < 0.001:
        # Assumes no negative values between -0.001 and 0.001
        return "<0.001"
    elif abs(x) < 1:
        return f"{x:.3f}"
    else:
        return f"{x:,.0f}"

## Configure

In [3]:
import datetime
import logging

datasets = [
    "jc_penney_products", 
    "online_boat_listings",
    "online_boat_listings_no_foreign_languages",
    "california_house_prices" 
]
## Change analysis dataset
dataset = datasets[0]

## Change whether we look at downsampled results (only for jc_penney_products)
downsamples = [0.2, 0.01, None]
downsample = downsamples[2]

## Change whether we look at normalised response
response_is_normalised = False

filter_out_replicates = False
filter_only_replicates = 4 # Can be int (4) or None, if int then only include results with this number of replicates, cannot be used with filter_out_replicates
filter_out_worst = False

log_response = False # Currently too difficult to interpret

which_ds_string = f"{dataset} at {datetime.datetime.now().isoformat()}"
print(which_ds_string)

jc_penney_products at 2025-03-11T11:33:24.593077


In [4]:

### Configure analysis
import sys


raw_metric = "rmse_test"
metric = raw_metric.removesuffix("_test")

pipeline_cols = ["preprocessers", "tokeniser", "featuriser", "model", "finetune"]
# Dont include finetune as we will alter the model name
hash_cols = ["preprocesser(s)", "tokeniser", "featuriser", "model"]
filter_worst_k_for = {
    "jc_penney_products": 0,
    "online_boat_listings": 0,
    "california_house_prices": 0
}
alpha = 0.05
### Configure plot saving
image_ext = "png"
OUTPUT_DIR = Path("notebook_output")
SAVE_HEIGHT = 400
SAVE_WIDTH = 500
table_dir = OUTPUT_DIR / dataset / "tables"
histogram_dir = OUTPUT_DIR / dataset / "1d_histograms"
boxplot_dir = OUTPUT_DIR / dataset / "1d_boxplots"
### Configure latex columns
latex_cols = ["preprocesser(s)", "tokeniser", "featuriser", "model", metric]
dataset_name_map = {
    "jc_penney_products": "jcp",
    "online_boat_listings": "obl",
}
###
histogram_dir.mkdir(exist_ok=True, parents=True)
table_dir.mkdir(exist_ok=True, parents=True)
boxplot_dir.mkdir(exist_ok=True, parents=True)

config_str = metric
if filter_out_replicates:
    config_str += "_no_replicates"
if filter_only_replicates is not None:
    config_str += f"_only_{filter_only_replicates}_replicates"
if filter_out_worst:
    config_str += f"_filter_worst_{filter_worst_k_for[dataset]}"
if downsample is not None:
    config_str += f"_downsample_{downsample}"
if response_is_normalised:
    config_str += "_normalise_response"

logger = logging.getLogger("analysis")
logger.setLevel(logging.DEBUG)
for handler in logger.handlers:
    logger.removeHandler(handler)
# Configure logging to file
log_file = OUTPUT_DIR / f"{dataset}/{config_str}.log"
log_file.parent.mkdir(parents=True, exist_ok=True)
# Remove all existing handlers
for handler in logger.handlers[:]:
    logger.removeHandler(handler)

file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.info(which_ds_string)

2025-03-11 11:33:24,614 - analysis - INFO - jc_penney_products at 2025-03-11T11:33:24.593077


In [5]:
print(f"Config: {config_str}")

Config: rmse_only_4_replicates


In [6]:

if filter_out_replicates and (filter_only_replicates is not None):
    raise ValueError("Cannot filter out replicates and only include replicates")

root_results = Path(f"results/")
if downsample is not None:
    root_results = root_results / f"downsample_{downsample}"
if response_is_normalised:
    root_results = root_results / f"normalised_response"

results = get_all_results(root_results / dataset, response_is_normalised=response_is_normalised, log_response=log_response)
logger.info(f"Found {len(results)} results for {dataset}")
train, val, test = load_dataset(dataset, seed=SEED)
mean = pd.concat([train, val])["label"].mean()
if "rmse" in metric:
    perf_of_mean = ((test["label"] - mean) ** 2).mean() ** 0.5
    if log_response:
        perf_of_mean = np.log(perf_of_mean)
    logger.info(f"Performance of mean with seed {SEED}: {perf_of_mean:.2f}")
else:
    raise NotImplementedError(f"Metric {metric} not implemented")

original_results = pd.DataFrame(results)
# Quickly strip suffix from metric name for latex tables/figures
original_results.rename(columns={raw_metric: metric}, inplace=True)
original_results.head()

2025-03-11 11:33:25,168 - analysis - INFO - Found 2584 results for jc_penney_products
2025-03-11 11:33:25,256 - analysis - INFO - Performance of mean with seed 97: 459.02


,preprocessers,tokeniser,featuriser,model,total_time,preprocess_time,model_time,rmse_train,rmse_val,rmse,...,mae_test,r2_train,r2_val,r2_test,medae_train,medae_val,medae_test,finetune,seed,preprocesser(s)
0,"[no_numbers_spacy, no_punctuation, no_stopword...",whitespace,bow_count,deep_linear,188.320256,144.192708,44.127548,340.860807,386.984260,355.809357,...,84.925031,0.592588,0.433308,0.398786,20.153707,31.814530,30.873307,False,1,"NNSC,NP,NSCC"
1,"[no_numbers_spacy, no_punctuation, no_stopword...",whitespace,bow_count,deep_linear,338.250530,174.657244,163.593286,323.709366,366.931999,355.322415,...,94.500459,0.634370,0.482099,0.400430,25.439191,40.348397,38.273909,False,97,"NNSC,NP,NSCC"
2,"[no_numbers_spacy, no_punctuation, no_stopword...",whitespace,glove_mean,xgboost,246.419280,245.773605,0.645675,152.374078,406.505359,442.135709,...,102.175853,0.918585,0.374693,0.071663,19.334696,22.967763,22.688130,False,1,"NNSC,NP,NSCC"
3,"[no_numbers_spacy, no_punctuation, no_stopword...",whitespace,glove_mean,xgboost,202.102430,200.952845,1.149585,154.237789,337.736983,425.236931,...,107.023927,0.916993,0.561234,0.141270,19.616468,24.034204,23.483477,False,97,"NNSC,NP,NSCC"
4,"[no_numbers_spacy, no_punctuation, no_stopword...",whitespace,glove_mean,xgboost,231.389905,230.918232,0.471673,238.010890,351.899757,426.812451,...,108.858736,0.820463,0.284517,0.134895,23.890877,24.821656,23.907604,False,2,"NNSC,NP,NSCC"


In [7]:
joined_data = pd.concat([train, val])
all_data = pd.concat([train, val, test])
mean_response = all_data["label"].mean()
standard_deviation_of_response = all_data["label"].std()
logger.info(f"Mean response (n data={len(all_data)}): {mean_response:.2f} +- {standard_deviation_of_response:.2f}")

2025-03-11 11:33:25,301 - analysis - INFO - Mean response (n data=13575): 127.72 +- 515.87


## Filter

In [8]:
filter_out = {
    "preprocessers": ["number"],
    "model": ["linear_regression"], # We filter out these results as they are just too bad, deep_linear models sorta do that anyway
    "tokeniser": [],
    "featuriser": [],
}
if filter_out_replicates:
    filter_out["seed"] = [1, 2, 3]

filtered_results = original_results.copy()
for feature, values in filter_out.items():
    if feature == "preprocessers":
        for value in values:
            filtered_results = filtered_results[~filtered_results["preprocessers"].apply(lambda x: any(value in y for y in x))]
    else:
        for value in values:
            filtered_results = filtered_results[filtered_results[feature] != value]
filtered_results.loc[filtered_results['finetune'], 'model'] = filtered_results.loc[filtered_results['finetune'], 'model'] + '_ft'



Analyse which seeds are missing in certain replicates if we want to increase sample size

In [9]:
replicates = filtered_results.groupby(hash_cols)["seed"].transform("count")
def seed_missing(seeds):
    return list(set([1,2,3,97]) - set(seeds))
missing_seeds = filtered_results[(1 < replicates) & (replicates < 4)].groupby(["preprocesser(s)", "tokeniser", "featuriser", "model"]).agg({"seed": lambda x: seed_missing(x)})
missing_seeds["seed"].value_counts()


seed
[2]        58
[1, 2]     55
[1]        42
[1, 3]      9
[97]        1
[97, 3]     1
Name: count, dtype: int64

In [10]:
if filter_only_replicates is not None:
    filtered_results = filtered_results[replicates == filter_only_replicates]

logger.info(f"Filtered to {len(filtered_results)} results")
filtered_results = filtered_results.sort_values(metric).reset_index(names="idx")
filtered_results["pipeline_hash"] = filtered_results[hash_cols].apply(lambda x: hash(tuple(x)), axis=1)

2025-03-11 11:33:37,105 - analysis - INFO - Filtered to 620 results


In [11]:
dupe_cols = ["pipeline_hash"]
if filter_only_replicates:
    dupe_cols.append("seed")
dupe_idx = filtered_results.duplicated(subset=dupe_cols, keep=False)
dupes = filtered_results[dupe_idx]
logger.info(f"Found {len(dupes)} duplicates")
display(dupes.sort_values("pipeline_hash").head(5))

2025-03-11 11:33:40,213 - analysis - INFO - Found 0 duplicates


,idx,preprocessers,tokeniser,featuriser,model,total_time,preprocess_time,model_time,rmse_train,rmse_val,...,r2_train,r2_val,r2_test,medae_train,medae_val,medae_test,finetune,seed,preprocesser(s),pipeline_hash


In [12]:
fig = px.box(filtered_results, y=metric, points="outliers")
def quantiles(df: pd.DataFrame):
    min_metric, max_metric = df[metric].min(), df[metric].max()
    q1_metric, q3_metric = df[metric].quantile(0.25), df[metric].quantile(0.75)
    median_metric = df[metric].median()
    return min_metric, q1_metric, median_metric, q3_metric, max_metric
min_metric, q1_metric, median_metric, q3_metric, max_metric = quantiles(filtered_results)
fig.update_yaxes(
    range=[
        filtered_results[metric].min()*0.9,
        (filtered_results[metric].median() + 1.5 * (filtered_results[metric].quantile(0.75) - filtered_results[metric].quantile(0.25)))*1.1
    ]
)
fig.add_hline(y=perf_of_mean, line_dash="dash", line_color="red", annotation_text=f"Mean baseline {perf_of_mean:.2f}")

fig.write_image(OUTPUT_DIR / f"{dataset}_{config_str}_boxplot.{image_ext}")

In [13]:
def threshold_histogram(df: pd.DataFrame, metric: str, min_val: float = None, max_val: float = None, mean_line: float = None):
    if min_val is not None:
        df = df[df[metric] >= min_val]
    if max_val is not None:
        df = df[df[metric] <= max_val]
    fig = px.histogram(df, x=metric, nbins=100)
    if mean_line is not None:
        fig.add_vline(x=mean_line, line_dash="dash", line_color="red", annotation_text=f"Mean baseline {mean_line:.2f}")
    return fig

fig = threshold_histogram(filtered_results, metric, min_metric, max_metric, perf_of_mean)
fig.show()

In [14]:
if filter_only_replicates:
    mean_rmse = filtered_results.groupby("pipeline_hash").agg({metric: "mean"})
    min_mean_rmse = mean_rmse[metric].min()
    max_mean_rmse = mean_rmse[metric].max()
    fig = threshold_histogram(mean_rmse, metric, min_mean_rmse, max_mean_rmse, perf_of_mean)
    fig.show()

# Timing of different groups of model

In [15]:
# Create histogram comparing timing of finetuned vs non-finetuned models
ft_pipelines = filtered_results[filtered_results["model"].str.contains("_ft", na=False)].copy()
bow_pipelines = filtered_results[filtered_results["featuriser"].isin(["bow_binary", "bow_count", "tf_idf"])].copy()
ft_times = ft_pipelines["total_time"].values
bow_times = bow_pipelines["total_time"].values

fig = go.Figure()
fig.add_trace(go.Histogram(x=ft_times, name="Finetuned Models", histnorm="percent", nbinsx=20))
fig.add_trace(go.Histogram(x=bow_times, name="BOW Models", histnorm="percent", nbinsx=20))

fig.update_layout(
    title="Training Time Distribution: Finetuned vs BOW Models",
    xaxis_title="Total Training Time (s)",
    yaxis_title="Count",
    barmode='overlay'
)

fig.update_traces(opacity=0.75)
fig.write_image(OUTPUT_DIR / f"{dataset}_{config_str}_timing_histogram.{image_ext}")
fig.show()


In [16]:
p_value, diff = permutation_test(bow_times, ft_times)

results = pd.DataFrame({
    "dataset": [dataset_name_map[dataset]],
    "mean-difference (BOW - FT)": [diff],
    "p_value": [p_value],
})

display(results)

latex_table = results.to_latex(float_format=simple_format, index=False)
latex_table = process_latex_table(latex_table)
with open(table_dir / f"{dataset}_{config_str}_time_permutation_test.tex", "w") as f:
    f.write(latex_table)


,dataset,mean-difference (BOW - FT),p_value
0,jcp,-2766.750859,0.00001


# Reduction from best to worst pipeline conditional on model

In [17]:
table_dir.mkdir(exist_ok=True, parents=True)
very_best = filtered_results[metric].sort_values().head(1).values[0]
very_worst = filtered_results[metric].sort_values().tail(1).values[0]
reduction_in_error = dict()
reduction_in_error["unconditional"] = (very_worst - very_best) / very_worst * 100
unique_models = filtered_results["model"].unique()
for model in unique_models:
    model_results = filtered_results[filtered_results["model"] == model]
    very_best = model_results[metric].sort_values().head(1).values[0]
    very_worst = model_results[metric].sort_values().tail(1).values[0]
    reduction_in_error[model] = (very_worst - very_best) / very_worst * 100

reduction_in_error = pd.Series(reduction_in_error, name="% Reduction in error").sort_index()
display(pd.DataFrame(reduction_in_error))

latex_table = reduction_in_error.to_latex(float_format=simple_format)
latex_table = process_latex_table(latex_table)

with open(table_dir / f"{dataset}_{config_str}_reduction_in_error.tex", "w") as f:
    f.write(latex_table)


,% Reduction in error
catboost,16.955141
deep_linear,20.123804
deep_linear_ft,21.808106
mlp,32.111567
mlp_ft,16.751690
resnet,14.900697
resnet_ft,16.869808
unconditional,36.970151
xgboost,36.970151


If the range of interest is compressed by outliers for plots we can filter them below; following config from above

In [18]:
# Filter top k
if filter_out_worst:
    k = filter_worst_k_for[dataset]
else: 
    k = 0
worst = filtered_results.sort_values(metric).tail(k)
results_df = filtered_results[~filtered_results.index.isin(worst.index)]

In [19]:
fig = threshold_histogram(results_df, metric, min_metric, max_metric, perf_of_mean)
fig.show()
fig.write_image(OUTPUT_DIR / f"{dataset}_{config_str}_filtered_{k}_histogram.{image_ext}")


# Record all performances in one CSV

In [20]:
_sorted = original_results[pipeline_cols + ["preprocesser(s)", "seed", metric]].sort_values(metric)
pipelines_cols_to_hash = pipeline_cols.copy()
pipelines_cols_to_hash.remove("preprocessers")
pipelines_cols_to_hash.append("preprocesser(s)")
_sorted["hash"] = _sorted.apply(lambda x: hash(tuple(x[pipelines_cols_to_hash])), axis=1)
_sorted.to_csv(table_dir / f"{config_str}_all_performances.csv")

# Look at best and worst performers

In [21]:
k = 10
if filter_out_replicates:
    num_replicates = 1
else:
    num_replicates = 4
unique_pipeline_cols = [col for col in pipeline_cols if col not in ["preprocessers", "finetune"]]
unique_pipeline_cols = ["preprocesser(s)"] + unique_pipeline_cols
unique_pipelines = filtered_results.groupby(unique_pipeline_cols)[[metric, "seed"]].agg(
 {
     metric: ["mean", "sem"],
     "seed" : "count"
 }
)

In [22]:
unique_pipelines_with_replicates = unique_pipelines[unique_pipelines["seed", "count"] == num_replicates]

In [23]:
if filter_out_replicates:
    columns_to_save = [[metric, "mean"]]
else:
    columns_to_save = [[metric, "mean"],[metric, "sem"]]
    
logger.info("Best performers")
best = unique_pipelines_with_replicates.sort_values((metric, "mean")).head(k)
latex_table = best[columns_to_save].to_latex(float_format=simple_format)
latex_table = process_latex_table(latex_table)
with open(table_dir / f"{config_str}_best_{k}.tex", "w") as f:
    f.write(latex_table)
display(best)
logger.info("Worst performers")
worst = unique_pipelines_with_replicates.sort_values((metric, "mean")).tail(k)
latex_table = worst[columns_to_save].to_latex(float_format=simple_format)
latex_table = process_latex_table(latex_table)
with open(table_dir / f"{config_str}_worst_{k}.tex", "w") as f:
    f.write(latex_table)
display(worst)


2025-03-11 11:34:37,638 - analysis - INFO - Best performers


rmse            \
                                                            mean       sem   
preprocesser(s) tokeniser  featuriser model                                  
NSN             bert       bert_mean  mlp_ft          328.380119  7.628759   
empty           bert       bert_mean  mlp_ft          329.863191  4.885979   
                whitespace tf_idf     mlp             334.181854  5.244016   
NSN             whitespace bow_count  mlp             334.871931  5.126242   
empty           treebank   tf_idf     mlp             335.467084  3.555399   
                bert       bert_mean  deep_linear_ft  336.610629  5.704093   
                stanford   tf_idf     mlp             337.163047  4.112963   
NSN,S           whitespace glove_mean resnet          338.273451  2.261563   
Lo,NSN          treebank   bow_binary mlp             338.431397  3.930526   
Le              bpe        tf_idf     mlp             338.845837  1.360006   

                                                      seed  
                                                     count  
preprocesser(s) tokeniser  featuriser model                 
NSN             bert       bert_mean  mlp_ft             4  
empty           bert       bert_mean  mlp_ft             4  
                whitespace tf_idf     mlp                4  
NSN             whitespace bow_count  mlp                4  
empty           treebank   tf_idf     mlp                4  
                bert       bert_mean  deep_linear_ft     4  
                stanford   tf_idf     mlp                4  
NSN,S           whitespace glove_mean resnet             4  
Lo,NSN          treebank   bow_binary mlp                4  
Le              bpe        tf_idf     mlp                4

2025-03-11 11:34:37,659 - analysis - INFO - Worst performers


rmse             seed
                                                         mean        sem count
preprocesser(s) tokeniser  featuriser model                                   
Lo              whitespace glove_mean mlp          402.479879   5.144282     4
Lo,NP,NSCC,S    stanford   glove_mean mlp          406.777786   1.275871     4
S               bert       bert_cls   xgboost      415.533966   8.531672     4
empty           whitespace glove_mean mlp          417.648696   2.988174     4
                stanford   glove_mean mlp          419.479988   2.687319     4
Lo              whitespace glove_mean deep_linear  432.014708   0.144993     4
NSCC,S          whitespace glove_mean xgboost      432.066470  18.725128     4
Le              stanford   glove_mean deep_linear  432.776268   0.249599     4
empty           whitespace glove_mean deep_linear  432.931561   0.518000     4
Le,Lo,NSCC      stanford   glove_mean mlp          441.580496  10.658201     4

Brief foray into looking at sem distribution - noting it is much more variable for finetuning

In [24]:
# Create histogram of standard errors, split by finetuned vs non-finetuned models
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = unique_pipelines_with_replicates.copy().reset_index()

# Get boolean mask for finetuned models
ft_mask = df['model'].str.contains('_ft')

# Create figure
fig = go.Figure()

# Add histograms
fig.add_trace(go.Histogram(
    x=df[~ft_mask][metric, 'sem'],
    name='Non-finetuned',
    opacity=0.5,
    nbinsx=20,
    histnorm="percent"
))

fig.add_trace(go.Histogram(
    x=df[ft_mask][metric, 'sem'], 
    name='Finetuned',
    opacity=0.5,
    nbinsx=20,
    histnorm="percent"
))

# Update layout
fig.update_layout(
    title='Distribution of Standard Errors by Model Type',
    xaxis_title='Standard Error of Mean',
    yaxis_title='Percentage',
    barmode='overlay',
    showlegend=True,
    template='plotly_white'
)

# Log summary statistics
logger.info("\nStandard Error Summary Statistics:")
logger.info(f"Non-finetuned mean: {df[~ft_mask][metric, 'sem'].mean():.2f}")
logger.info(f"Finetuned mean: {df[ft_mask][metric, 'sem'].mean():.2f}")

2025-03-11 11:34:47,655 - analysis - INFO - 
Standard Error Summary Statistics:
2025-03-11 11:34:47,657 - analysis - INFO - Non-finetuned mean: 5.05
2025-03-11 11:34:47,659 - analysis - INFO - Finetuned mean: 8.04


# Confirm coverage of component choices

In [25]:
def count_unique_values(df: pd.DataFrame, feature: str):
    return df[feature].value_counts().to_dict()

def count_unique_values_of_set(df: pd.DataFrame, feature: str):
    counts = {}
    for row in df.itertuples():
        in_the_bag = getattr(row, feature)
        for feature_value in in_the_bag:
            counts[feature_value] = counts.get(feature_value, 0) + 1
    return counts

def create_pipeline_counts_table(df: pd.DataFrame):
    tables = []
    # Get counts for each pipeline step
    counts = {}
    counts["preprocessers"] = count_unique_values_of_set(df, "preprocessers")
    counts["tokeniser"] = count_unique_values(df, "tokeniser") 
    counts["featuriser"] = count_unique_values(df, "featuriser")
    counts["model"] = count_unique_values(df, "model")
    
    for feature in ["preprocessers", "tokeniser", "featuriser", "model"]:
        counts_df = pd.DataFrame({feature: pd.Series(counts[feature])})
        tables.append(counts_df)
        latex_table = counts_df.to_latex(
            float_format=simple_format,
            na_rep='',
            header=False,
            escape=False
        )
        latex_table = latex_table.replace("_", "-")

        # Save to file
        table_path = table_dir / f"{config_str}_pipeline_counts_{feature}.tex"
        with open(table_path, 'w') as f:
            f.write(latex_table)
        
    return tables

# Generate and display the table
pipeline_counts = create_pipeline_counts_table(results_df)
for table in pipeline_counts:
    display(table)


,preprocessers
no_punctuation,196
stem,164
no_stopwords_nltk,120
empty,168
lowercase,220
lemmatize,156
no_stopwords_spacy,140


,tokeniser
bert,260
stanford,120
whitespace,116
bpe,72
treebank,52


,featuriser
tf_idf,116
bow_binary,112
bert_cls,104
bow_count,100
bert_mean,96
glove_mean,92


,model
catboost,140
deep_linear,124
xgboost,112
mlp,108
resnet,80
resnet_ft,24
mlp_ft,16
deep_linear_ft,16


# Look at spread associated with each component

In [26]:
def histogram_on_feature(
        df: pd.DataFrame, 
        feature: str, 
        metric: str, 
        threshold: float = None, 
        histogram: bool = True,
        mean_line: float = None,
        **kwargs
    ):
    if threshold is not None:
        df = df[df[metric] < threshold]
    unique_features = df[feature].unique()
    unique_features = sorted(unique_features)
    if histogram:
        fig = make_subplots(rows=len(unique_features), cols=1, shared_xaxes=True, subplot_titles=unique_features)
    else:
        fig = make_subplots(rows=1, cols=len(unique_features), shared_yaxes=True, subplot_titles=unique_features)
    for i, feature_value in enumerate(unique_features):
        feature_data = df[df[feature] == feature_value][metric]
        if histogram:
            fig.add_trace(
                go.Histogram(
                x=feature_data, 
                name=feature_value, 
                texttemplate="%{y:.2f}",
                histnorm="percent",
                marker_color="gray",
                **kwargs
            ), 
                row=i+1, col=1
            )
        else:
            fig.add_trace(
                go.Box(
                    y=feature_data,
                    name=feature_value,
                    marker_color="gray",
                    **kwargs
                ),
                row=1, col=i+1
            )
    if mean_line is not None:
        if histogram:
            fig.add_vline(x=mean_line, line_dash="dash", line_color="red")
        else:
            fig.add_hline(y=mean_line, line_dash="dash", line_color="red")
    if histogram:
        fig.update_layout(height=200 * len(unique_features))
    else:
        fig.update_layout(width=200 * len(unique_features))
    fig.update_layout(title=f"Distribution of {metric} by {feature}")
    fig.update_layout(showlegend=False)
    return fig

def histogram_on_set(
        df: pd.DataFrame, 
        feature: str, 
        metric: str, 
        threshold: float = None,
        histogram: bool = True,
        mean_line: float = None,
        **kwargs
    ):
    if threshold is not None:
        df = df[df[metric] < threshold]
    bags = defaultdict(list)
    for row in df.itertuples():
        bag = getattr(row, feature)
        for feature_value in bag:
            bags[feature_value].append(getattr(row, metric))
        if len(bag) == 0:
            bags["Empty"].append(getattr(row, metric))
    bags = dict(sorted(bags.items()))
    if histogram:
        fig = make_subplots(rows=len(bags), cols=1, shared_xaxes=True, subplot_titles=list(bags.keys()))
    else:
        fig = make_subplots(rows=1, cols=len(bags), shared_yaxes=True, subplot_titles=list(bags.keys()))
    for i, (feature_value, values) in enumerate(bags.items()):
        if histogram:
            fig.add_trace(
                go.Histogram(
                    x=values,
                    name=feature_value,
                    texttemplate="%{y:.2f}",
                    histnorm="percent",
                    marker_color="gray",
                    **kwargs
                ),
                row=i+1, col=1
            )
        else:
            fig.add_trace(
                go.Box(
                    y=values,
                    name=feature_value,
                    marker_color="gray",
                    **kwargs
                ),
                row=1, col=i+1
            )
    if mean_line is not None:
        if histogram:
            fig.add_vline(x=mean_line, line_dash="dash", line_color="red")
        else:
            fig.add_hline(y=mean_line, line_dash="dash", line_color="red")
    if histogram:
        fig.update_layout(height=200 * len(bags))
    else:
        fig.update_layout(width=200 * len(bags))
    fig.update_layout(title=f"Distribution of {metric} for {feature}")
    fig.update_layout(showlegend=False)
    return fig


fig = histogram_on_set(results_df, "preprocessers", metric, mean_line=perf_of_mean)
fig.write_image(histogram_dir / f"{config_str}_preprocessers_histogram.{image_ext}")
for feature in ["tokeniser", "featuriser", "model"]:
    fig = histogram_on_feature(results_df, feature, metric, mean_line=perf_of_mean)
    fig.write_image(histogram_dir / f"{config_str}_{feature}_histogram.{image_ext}")


In [27]:
fig = histogram_on_set(results_df, "preprocessers", metric, histogram=False, mean_line=perf_of_mean)
# fig.show()
# fig.write_html(OUTPUT_DIR / f"{dataset}_{metric}_filtered_{k}_preprocessers_boxplot.html")
fig.write_image(boxplot_dir / f"{config_str}_preprocessers_boxplot.{image_ext}")

for feature in ["tokeniser", "featuriser", "model"]:
    fig = histogram_on_feature(results_df, feature, metric, histogram=False, mean_line=perf_of_mean)
    # fig.show()
    # fig.write_html(OUTPUT_DIR / f"{dataset}_{metric}_filtered_{k}_{feature}_boxplot.html")
    fig.write_image(boxplot_dir / f"{config_str}_{feature}_boxplot.{image_ext}")


# Develop filtering/conditioning logic

In [28]:
def condition_on(
        equals_conditions: list[tuple[str, str]], 
        not_equals_conditions: list[tuple[str, str]], 
        df: pd.DataFrame
    ):
    """
    A condition is a tuple of (column, value)

    We want to filter the dataframe to only include rows where the equals 
    conditions are true AND the not equals conditions are false.
    """
    subset = df.copy()  
    for condition in equals_conditions:
        idx = subset[condition[0]].str.contains(condition[1])
        subset = subset.loc[idx, :]
    for condition in not_equals_conditions:
        idx = subset[condition[0]] != condition[1]
        subset = subset.loc[idx, :]
    return subset

def condition_featuriser(df: pd.DataFrame, option: str):
    condition = [("featuriser", option)]
    return condition_on(condition, [], df)

def condition_not_featuriser(df: pd.DataFrame, option: str):
    condition = [("featuriser", option)]
    return condition_on([], condition, df)

featurisers = ["glove_mean", "bow_count", "bow_binary", "tf_idf", "bert_cls", "bert_mean"]
conditioned_on = {}
conditioned_not_on = {}

for condition in featurisers:
    conditioned_on[condition] = condition_featuriser(filtered_results, condition)
    conditioned_not_on[condition] = condition_not_featuriser(filtered_results, condition)

# BoW for DL

In [30]:
if filter_only_replicates:
    to_consider_rmse = filtered_results.groupby(["preprocesser(s)", "tokeniser", "featuriser", "model"]).agg({metric: ["mean", "sem"]}).reset_index()
    to_consider_rmse.columns = ["preprocesser(s)", "tokeniser", "featuriser", "model", f"{metric}", f"{metric}_sem"]
    title = "dist of mean RMSE for each pipeline"
else:
    to_consider_rmse = filtered_results
    title = "dist of RMSE for each pipeline"

In [31]:
dl_models = ["mlp", "resnet", "deep_linear"]
gbdt_models = ["catboost", "xgboost"]
bow_models = ["bow_count", "bow_binary", "tf_idf"]
ignore_glove = to_consider_rmse["featuriser"] != "glove_mean"
bow_comparison_df = to_consider_rmse[ignore_glove]
dl_models = dl_models
ft_dl_models = [f"{x}_ft" for x in dl_models]
is_dl_model = bow_comparison_df["model"].isin(dl_models)
is_ft_dl_model = bow_comparison_df["model"].isin(ft_dl_models)
is_bow_model = bow_comparison_df["featuriser"].isin(bow_models)
is_gbdt_model = bow_comparison_df["model"].isin(gbdt_models)

bow_dl_results = bow_comparison_df[
    is_dl_model & is_bow_model
]
bow_ft_dl_results = bow_comparison_df[
    is_ft_dl_model & is_bow_model
]
bow_gbdt_results = bow_comparison_df[
    is_gbdt_model & is_bow_model
]
not_bow_dl_results = bow_comparison_df[
    is_dl_model & ~is_bow_model
]
not_bow_ft_dl_results = bow_comparison_df[
    is_ft_dl_model & ~is_bow_model
]
bow_gbdt_results["model_type"] = "GBDT w/ BoW"
bow_dl_results["model_type"] = "DL w/ BoW"
bow_ft_dl_results["model_type"] = "DL w/ BoW + Finetuning" # Wait this shouldnt be possible?
not_bow_dl_results["model_type"] = "DL w/o BoW"
not_bow_ft_dl_results["model_type"] = "DL w/o BoW + Finetuning"

comparison = pd.concat([
    bow_dl_results, 
    not_bow_dl_results, 
    bow_ft_dl_results, 
    not_bow_ft_dl_results,
    bow_gbdt_results
])
fig = px.box(
    comparison,
    x="model_type",
    y=metric,
    points="outliers",
    title=title
)
fig.add_hline(y=perf_of_mean, line_dash="dash", line_color="gray")
fig.write_image(
    OUTPUT_DIR / dataset / "1d_boxplots" / f"{config_str}_bow_dl_gbdt.{image_ext}",
    height=SAVE_HEIGHT,
    width=SAVE_WIDTH,
    scale=2
)


/tmp/ipykernel_56187/3578367382.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_56187/3578367382.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_56187/3578367382.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykerne

# DL w/ Finetuning vs w/out

Look into whether we have graphically noticable differences between pipelines that had BERT featurisation using and not using finetuning

In [32]:
plot_version = 4

In [33]:
unique_models = filtered_results["model"].unique()
unique_models 

array(['xgboost', 'mlp_ft', 'deep_linear_ft', 'mlp', 'catboost', 'resnet',
       'deep_linear', 'resnet_ft'], dtype=object)

In [34]:
if False:
    to_consider_rmse = filtered_results.groupby(["preprocesser(s)", "tokeniser", "featuriser", "model"]).agg({metric: ["mean", "sem"]}).reset_index()
    to_consider_rmse.columns = ["preprocesser(s)", "tokeniser", "featuriser", "model", f"{metric}", f"{metric}_sem"]
else:
    to_consider_rmse = filtered_results

In [35]:
using_ft = condition_on([("model", "ft")], [], to_consider_rmse)
with_mlp = to_consider_rmse["model"] == "mlp"
with_linear = to_consider_rmse["model"] == "deep_linear"
with_resnet = to_consider_rmse["model"] == "resnet"
using_dl = with_mlp | with_linear | with_resnet
with_bert_featurizer = (to_consider_rmse["featuriser"] == "bert_cls") | (to_consider_rmse["featuriser"] == "bert_mean")
with_bow_featurizer = (to_consider_rmse["featuriser"] == "bow_count") | (to_consider_rmse["featuriser"] == "bow_binary") | (to_consider_rmse["featuriser"] == "tf_idf")
with_glove_featurizer = (to_consider_rmse["featuriser"] == "glove_mean")

using_dl_no_ft = using_dl & with_bert_featurizer
incompatible_dl = using_dl & ~with_bert_featurizer
bert_no_dl = ~using_dl & with_bert_featurizer
using_dl_w_glove = using_dl & with_glove_featurizer
dl_w_bow = using_dl & with_bow_featurizer

subsampled = {}

subsampled["using_ft"] = using_ft
subsampled["using_dl_no_ft"] = to_consider_rmse[using_dl_no_ft]
subsampled["using_dl_no_ft_incompatible"] = to_consider_rmse[incompatible_dl]
subsampled["using_dl_w_bow"] = to_consider_rmse[dl_w_bow]
subsampled["using_dl_w_glove"] = to_consider_rmse[using_dl_w_glove]
subsampled["using_bert_no_dl"] = to_consider_rmse[bert_no_dl]
subsampled["using_bow"] = to_consider_rmse[with_bow_featurizer]

compare_dl_ft = pd.DataFrame({
    "Model Type": ["DL + BERT w/ Finetuning", "DL + BERT w/o Finetuning", "DL + BoW"],
    "Mean": [
        subsampled["using_ft"][metric].mean(), 
        subsampled["using_dl_no_ft"][metric].mean(), 
        subsampled["using_dl_w_bow"][metric].mean()
    ],
    "Standard Error": [
        subsampled["using_ft"][metric].sem(), 
        subsampled["using_dl_no_ft"][metric].sem(), 
        subsampled["using_dl_w_bow"][metric].sem()
    ]
}).round(0)

latex_table = compare_dl_ft.to_latex(
    index=False,
    float_format=simple_format
)
latex_table = process_latex_table(latex_table)
with open(table_dir / f"{config_str}_compare_dl_ft.tex", "w") as f:
    f.write(latex_table)

subsampled["using_ft"]["model_type"] = "BERT+DL w/ Finetuning"
subsampled["using_dl_no_ft"]["model_type"] = "BERT+DL w/o Finetuning"
subsampled["using_dl_no_ft_incompatible"]["model_type"] = "Not BERT+DL"
subsampled["using_bert_no_dl"]["model_type"] = "BERT w/o DL"
subsampled["using_dl_w_bow"]["model_type"] = "BoW+DL"
subsampled["using_dl_w_glove"]["model_type"] = "GloVe+DL"
subsampled["using_bow"]["model_type"] = "BoW"

# Can also group by pipeline_hash and model_type - to compare main effects, not thought it through though so will just use raw data
grouped_subsampled = {}
for sampled in subsampled:
    grouped_subsampled[sampled] = subsampled[sampled].groupby(["pipeline_hash", "model_type"]).agg({metric: "mean"}).reset_index()

if plot_version == 5:
    to_analyze = grouped_subsampled
else:
    to_analyze = subsampled

to_combine = [
    to_analyze["using_ft"], 
    to_analyze["using_dl_no_ft"]
]
if plot_version == 2:
    to_combine.append(to_analyze["using_dl_no_ft_incompatible"])
if plot_version == 3:
    to_combine.append(to_analyze["using_dl_w_bow"])
    to_combine.append(to_analyze["using_dl_w_glove"])
    to_combine.append(to_analyze["using_bert_no_dl"])
if plot_version == 4 or plot_version == 5:
    to_combine.append(to_analyze["using_bow"])
combined = pd.concat(to_combine)


fig = px.box(
    combined,
    y=metric,
    x="model_type",
    points="outliers"
)
fig.add_hline(y=perf_of_mean, line_dash="dash", line_color="gray")
savepath = OUTPUT_DIR / dataset / "1d_boxplots" / f"{config_str}_compare_dl_ft.{image_ext}"
if plot_version == 2:
    savepath = OUTPUT_DIR / dataset / "1d_boxplots" / f"{config_str}_compare_dl_ft_incompatible.{image_ext}"
if plot_version == 3:
    savepath = OUTPUT_DIR / dataset / "1d_boxplots" / f"{config_str}_compare_dl_ft_bow.{image_ext}"
if plot_version == 4:
    savepath = OUTPUT_DIR / dataset / "1d_boxplots" / f"{config_str}_compare_dl_ft_bow_v4.{image_ext}"
if plot_version == 5:
    savepath = OUTPUT_DIR / dataset / "1d_boxplots" / f"{config_str}_compare_dl_ft_bow_v5.{image_ext}"
fig.write_image(
    savepath,
    height=SAVE_HEIGHT,
    width=SAVE_WIDTH,
    scale=2
)


/tmp/ipykernel_56187/2484556661.py:49: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_56187/2484556661.py:50: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_56187/2484556661.py:51: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykerne

In [36]:
using_gbdt_w_bow = to_consider_rmse[
    ~using_dl & with_bow_featurizer
]
using_gbdt_w_bow["model_type"] = "BoW + GBDT"
using_gbdt_w_glove = to_consider_rmse[
    ~using_dl & with_glove_featurizer
]
using_gbdt_w_glove["model_type"] = "GloVe + GBDT"
using_gbdt_w_bert = to_consider_rmse[
    ~using_dl & with_bert_featurizer
]
using_gbdt_w_bert["model_type"] = "BERT + GBDT"

subsampled["using_gbdt_w_bow"] = using_gbdt_w_bow
subsampled["using_gbdt_w_glove"] = using_gbdt_w_glove
subsampled["using_gbdt_w_bert"] = using_gbdt_w_bert

if plot_version == 5:
    grouped_subsampled["using_gbdt_w_bow"] = using_gbdt_w_bow.groupby(["pipeline_hash", "model_type"]).agg({metric: "mean"}).reset_index()
    grouped_subsampled["using_gbdt_w_glove"] = using_gbdt_w_glove.groupby(["pipeline_hash", "model_type"]).agg({metric: "mean"}).reset_index()
    grouped_subsampled["using_gbdt_w_bert"] = using_gbdt_w_bert.groupby(["pipeline_hash", "model_type"]).agg({metric: "mean"}).reset_index()

all_combined = pd.concat([
    combined, to_analyze["using_gbdt_w_bow"]
])

if plot_version == 3 or plot_version == 4 or plot_version == 5:
    all_combined = pd.concat([
        all_combined, 
        to_analyze["using_gbdt_w_glove"], 
        to_analyze["using_gbdt_w_bert"]
    ])

fig = px.box(
    all_combined,
    y=metric,
    x="model_type",
    points="outliers"
)
fig.add_hline(y=perf_of_mean, line_dash="dash", line_color="gray")
savepath = OUTPUT_DIR / dataset / "1d_boxplots" / f"{config_str}_compare_dl_ft_and_bow.{image_ext}"
if plot_version == 2:
    savepath = OUTPUT_DIR / dataset / "1d_boxplots" / f"{config_str}_compare_dl_ft_and_bow_incompatible.{image_ext}"
if plot_version == 3:
    savepath = OUTPUT_DIR / dataset / "1d_boxplots" / f"{config_str}_compare_dl_ft_and_bow_w_wout_dl.{image_ext}"
if plot_version == 4:
    savepath = OUTPUT_DIR / dataset / "1d_boxplots" / f"{config_str}_compare_dl_ft_and_bow_v4.{image_ext}"
if plot_version == 5:
    savepath = OUTPUT_DIR / dataset / "1d_boxplots" / f"{config_str}_compare_dl_ft_and_bow_v5.{image_ext}"
fig.write_image(
    savepath,
    height=SAVE_HEIGHT,
    width=SAVE_WIDTH,
    scale=2
)

/tmp/ipykernel_56187/3240004188.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_56187/3240004188.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_56187/3240004188.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



For BERT featurization to reach the performance of BoW (with or without DL, surprisingly DL performing strongly with BoW) then we must use finetuning

## Testing for equivalence to validate reading of graph

In [37]:
pairs = [
    (("ft", to_analyze["using_ft"]), ("no_ft", to_analyze["using_dl_no_ft"])),
    (("ft", to_analyze["using_ft"]), ("bow", to_analyze["using_bow"])),
    (("no_ft", to_analyze["using_dl_no_ft"]), ("bow", to_analyze["using_bow"])),
]
rows = []
for (a, b) in pairs:
    a_name, a_df = a
    b_name, b_df = b
    p_val, diff = permutation_test(a_df[metric], b_df[metric])
    row = {
        'comparison': f'{a_name} vs {b_name}',
        'mean_difference': diff,
        'p_value': p_val
    }
    rows.append(row)
results = pd.DataFrame(rows)
table = results.to_latex(
    index=False,
    float_format=simple_format
)
table = process_latex_table(table)
savepath = OUTPUT_DIR / dataset / "tables" / f"{config_str}_compare_dl_w_wout_ft_and_bow.tex"
if plot_version == 5:
    savepath = OUTPUT_DIR / dataset / "tables" / f"{config_str}_compare_dl_w_wout_ft_and_bow_v5.tex"
with open(savepath, "w") as f:
    f.write(table)
display(results)
print(f"Saved to {savepath}")

,comparison,mean_difference,p_value
0,ft vs no_ft,-30.014917,0.00001
1,ft vs bow,5.388124,0.02262
2,no_ft vs bow,35.403041,0.00001


Saved to notebook_output/jc_penney_products/tables/rmse_only_4_replicates_compare_dl_w_wout_ft_and_bow.tex


## Do this blocked pairwise on the seeds
Most powerful approach - see if this gives substantially different results

In [38]:
# Create a dictionary similar to subsampled but with seed filtering
subsampled_by_seed = {}
seeds = subsampled["using_ft"]["seed"].unique()
for seed in seeds:
    if seed not in subsampled_by_seed:
        subsampled_by_seed[seed] = {}
    # Group by key and get mean RMSE for each seed
    for key, df in subsampled.items():
        df_seed = df[df["seed"] == seed]
        subsampled_by_seed[seed][key] = df_seed[metric].values


In [39]:
subsampled_by_seed.keys()

dict_keys([97, 2, 1, 3])

In [40]:
def blocked_permutation_test(nested_dict, key1, key2, n_permutations=10000):
    """
    Perform a blocked permutation test comparing two groups within blocks.
    
    Args:
        nested_dict: Dictionary of dictionaries where first level is blocks and second level is groups, final level should be numpy arrays
        key1: First group to compare
        key2: Second group to compare 
        n_permutations: Number of permutations to perform
        
    Returns:
        p_value: P-value from permutation test
        observed_diff: Observed difference in means between groups
    """
    # Calculate observed difference in means
    group_1 = []
    group_2 = []
    blocks = nested_dict.keys()
    for block in blocks:
        group_1.append(nested_dict[block][key1])
        group_2.append(nested_dict[block][key2])
    observed_diff = np.mean(np.concatenate(group_1)) - np.mean(np.concatenate(group_2))
    
    # Perform permutations within each block
    count = 0
    for _ in range(n_permutations):
        perm_group_1 = []
        perm_group_2 = []
        for block in blocks:
            # Get values for both groups in this block
            vals1 = nested_dict[block][key1]
            vals2 = nested_dict[block][key2]
            all_vals = np.concatenate([vals1, vals2])
            
            # Randomly permute and split back into groups
            np.random.shuffle(all_vals)
            perm1 = all_vals[:len(vals1)]
            perm2 = all_vals[len(vals1):]
            perm_group_1.append(perm1)
            perm_group_2.append(perm2)
            
        # Calculate overall permuted difference
        perm_mean_diff = np.mean(np.concatenate(perm_group_1)) - np.mean(np.concatenate(perm_group_2))
        if abs(perm_mean_diff) >= abs(observed_diff):
            count += 1
            
    p_value = (count + 1) / (n_permutations + 1)
    
    return p_value, observed_diff

In [41]:

# Create synthetic data from normal distribution to demonstrate importance of blocked testing
# Create two blocks with different means but same treatment effect
n_per_group = 20
blocked_vals = {
    "block1": {},
    "block2": {}
}
blocked_vals["block1"]["control"] = np.random.normal(loc=100, scale=10, size=n_per_group)
blocked_vals["block2"]["control"] = np.random.normal(loc=150, scale=10, size=n_per_group)

blocked_vals["block1"]["treatment"] = np.random.normal(loc=105, scale=10, size=n_per_group) # Effect size = 5
blocked_vals["block2"]["treatment"] = np.random.normal(loc=155, scale=10, size=n_per_group)  

# Naive t-test ignoring blocks
all_control = np.concatenate([blocked_vals["block1"]["control"], blocked_vals["block2"]["control"]])
all_treatment = np.concatenate([blocked_vals["block1"]["treatment"], blocked_vals["block2"]["treatment"]])

p_val, observed_diff = permutation_test(all_treatment, all_control)
print(f"Naive permutation test p-value: {p_val:.3f}")
print(f"Observed difference: {observed_diff:.3f}")


p_val, observed_diff = blocked_permutation_test(blocked_vals, "treatment", "control")
print(f"Blocked permutation test p-value: {p_val:.3f}")
print(f"Observed difference: {observed_diff:.3f}")


Naive permutation test p-value: 0.383
Observed difference: 5.372
Blocked permutation test p-value: 0.037
Observed difference: 5.372


In [42]:
name_to_col_map = {
    "ft": "using_ft",
    "no_ft": "using_dl_no_ft",
    "bow": "using_bow",
}
p_vals = []
for a,b in pairs:
    a_name, a_df = a
    b_name, b_df = b    
    p_val, observed_diff = blocked_permutation_test(subsampled_by_seed, name_to_col_map[a_name], name_to_col_map[b_name])
    print(f"Blocked permutation test p-value for {a_name} vs {b_name}: {p_val:.4f}")
    print(f"Observed difference: {observed_diff:.4f}")
    p_vals.append(p_val)


Blocked permutation test p-value for ft vs no_ft: 0.0001
Observed difference: -30.0149
Blocked permutation test p-value for ft vs bow: 0.0217
Observed difference: 5.3881
Blocked permutation test p-value for no_ft vs bow: 0.0001
Observed difference: 35.4030


In [43]:
# Apply Benjamini-Hochberg correction to p-values
p_vals = np.array(p_vals)
n_tests = len(p_vals)
sorted_p_idx = np.argsort(p_vals)
sorted_p_vals = p_vals[sorted_p_idx]

# Calculate BH critical values
bh_values = np.arange(1, n_tests + 1) * alpha / n_tests

# Find largest p-value that is less than corresponding BH critical value
is_significant = sorted_p_vals <= bh_values
if np.any(is_significant):
    max_sig_idx = np.where(is_significant)[0][-1]
    adjusted_alpha = bh_values[max_sig_idx]
    logger.info(f"\nAfter Benjamini-Hochberg correction (alpha={alpha}):")
    logger.info(f"Adjusted significance threshold: {adjusted_alpha:.4f}")
    logger.info(f"First {max_sig_idx+1} comparisons are significant")
else:
    logger.info("\nNo comparisons significant after Benjamini-Hochberg correction")

# Plot p-values against their rank with BH line
fig = go.Figure()

# Plot points
rank = np.arange(1, len(sorted_p_vals) + 1)
fig.add_trace(
    go.Scatter(
        x=rank,
        y=sorted_p_vals,
        mode='markers',
        name='P-values',
        marker=dict(color='blue')
    )
)

# Plot BH line
fig.add_trace(
    go.Scatter(
        x=rank,
        y=bh_values,
        mode='lines',
        name=f'BH line (α={alpha})',
        line=dict(color='red', dash='dash')
    )
)

# Update layout
fig.update_layout(
    title='P-values vs Benjamini-Hochberg Critical Values',
    xaxis_title='Rank',
    yaxis_title='P-value',
    showlegend=True,
    template='plotly_white'
)

fig.show()




2025-03-11 11:37:56,509 - analysis - INFO - 
After Benjamini-Hochberg correction (alpha=0.05):
2025-03-11 11:37:56,511 - analysis - INFO - Adjusted significance threshold: 0.0500
2025-03-11 11:37:56,512 - analysis - INFO - First 3 comparisons are significant


# bert_cls vs glove: Pairwise comparison

In [44]:
cols_of_interest = latex_cols + ["seed", "preprocessers"]
glove_performances = conditioned_on["glove_mean"][cols_of_interest]
bert_performances = pd.concat([conditioned_on["bert_cls"], conditioned_on["bert_mean"]])[cols_of_interest]

# def filter_out_stemming(df):
#     return df[df["preprocessers"].apply(lambda x: "stem" not in x)]

# glove_performances = filter_out_stemming(glove_performances)
# bert_performances = filter_out_stemming(bert_performances)

shared_pipelines = glove_performances.merge(
    bert_performances,
    on=[
        "preprocesser(s)",  # Unsure if we should also merge on preprocesser(s) as it fragments results, but it strengthens test
        "model",
        "seed"
    ],
    suffixes=["_glove", "_bert"],
    how="inner"
)
shared_pipelines[metric+"_diff"] = shared_pipelines[metric+"_bert"] - shared_pipelines[metric+"_glove"]
shared_pipelines = shared_pipelines.sort_values(metric+"_diff")
print(shared_pipelines.shape)
display(shared_pipelines)

(28, 12)


,preprocesser(s),tokeniser_glove,featuriser_glove,model,rmse_glove,seed,preprocessers_glove,tokeniser_bert,featuriser_bert,rmse_bert,preprocessers_bert,rmse_diff
15,Lo,whitespace,glove_mean,mlp,411.519391,1,[lowercase],bert,bert_cls,369.499701,[lowercase],-42.019690
14,Lo,whitespace,glove_mean,mlp,409.706060,2,[lowercase],bert,bert_cls,370.877050,[lowercase],-38.829009
26,empty,whitespace,glove_mean,deep_linear,434.439971,2,[empty],bert,bert_cls,397.464240,[empty],-36.975731
27,empty,whitespace,glove_mean,deep_linear,434.439971,2,[empty],bert,bert_mean,397.869226,[empty],-36.570745
20,empty,whitespace,glove_mean,deep_linear,432.206056,97,[empty],bert,bert_mean,395.802287,[empty],-36.403769
17,Lo,whitespace,glove_mean,deep_linear,432.010309,97,[lowercase],bert,bert_mean,395.802287,[lowercase],-36.208022
21,empty,whitespace,glove_mean,deep_linear,432.303724,3,[empty],bert,bert_cls,396.243567,[empty],-36.060157
24,empty,whitespace,glove_mean,deep_linear,432.776494,1,[empty],bert,bert_cls,396.818474,[empty],-35.958020
19,empty,whitespace,glove_mean,deep_linear,432.206056,97,[empty],bert,bert_cls,397.011463,[empty],-35.194593
22,empty,whitespace,glove_mean,deep_linear,432.303724,3,[empty],bert,bert_mean,397.644649,[empty],-34.659074


In [45]:
from itertools import product

def binary_counter_gen(len):
    for mask in product([True, False], repeat=len):
        yield np.array(mask)


In [46]:
for sample in binary_counter_gen(3):
    print(sample)

[ True  True  True]
[ True  True False]
[ True False  True]
[ True False False]
[False  True  True]
[False  True False]
[False False  True]
[False False False]


In [47]:


def paired_permutation_test(a, b, n_permutations=10000, random=True):
    """
    Tests that the mean difference between two paired samples is significantly different from 0.

    Do so by shuffling membership of a and b and checking how often the mean difference is greater 
    than the observed difference.
    """

    if len(a) != len(b):
        raise ValueError("a and b must have the same length")
    max_num_perms = 2**len(a)
    if n_permutations > max_num_perms:
        if random:
            logger.warning(f"n_permutations ({n_permutations:.0e}) is greater than the maximum number of permutations ({max_num_perms:.0e}). This may give misleading results.")
        else:
            logger.warning(f"n_permutations ({n_permutations:.0e}) is greater than the maximum number of permutations ({max_num_perms:.0e}). Flooring to {max_num_perms}.")
            n_permutations = max_num_perms
    elif n_permutations < max_num_perms and not random:
        logger.warning(f"n_permutations ({n_permutations:.0e}) is less than the maximum number of permutations ({max_num_perms:.0e}). This may give misleading results. As such we will use all permutations.")
        n_permutations = max_num_perms
    a_copy = a.copy()
    b_copy = b.copy()
    observed_diff = (a_copy - b_copy).mean()
    more_extreme_than_observed = 0.
    if not random:
        binary_counter = iter(binary_counter_gen(len(a)))
    for i in range(n_permutations):
        new_a = a_copy.copy()
        new_b = b_copy.copy()
        if random:
            mask = np.random.choice([True, False], size=len(a), replace=True)
            new_a[mask] = b_copy[mask]
            new_b[mask] = a_copy[mask]
            mean_diff = (new_a - new_b).mean()
            if abs(mean_diff) >= abs(observed_diff):
                more_extreme_than_observed += 1
        else:
            mask = next(binary_counter)
            new_a[mask] = b_copy[mask]
            new_b[mask] = a_copy[mask]
            mean_diff = (new_a - new_b).mean()
            if abs(mean_diff) >= abs(observed_diff):
                more_extreme_than_observed += 1
    return (more_extreme_than_observed+1) / (n_permutations+1), observed_diff

In [48]:
# Test that paired permutation test works correctly on identical data
normal_sample = np.random.normal(size=100)
same_normal_sample = np.random.normal(size=100)
p_val, observed_diff = paired_permutation_test(normal_sample, same_normal_sample, random=True)
logger.info(f"\nPaired permutation test on identical data:")
logger.info(f"Observed difference: {observed_diff:.2f}")
logger.info(f"P-value: {p_val}")
logger.info(f"Statistically {'different' if p_val < alpha else 'not different'} from 0 at {alpha} significance")


2025-03-11 11:38:10,263 - analysis - INFO - 
Paired permutation test on identical data:
2025-03-11 11:38:10,264 - analysis - INFO - Observed difference: 0.03
2025-03-11 11:38:10,265 - analysis - INFO - P-value: 0.8401159884011599
2025-03-11 11:38:10,266 - analysis - INFO - Statistically not different from 0 at 0.05 significance


In [49]:
p_val, observed_diff = paired_permutation_test(
    shared_pipelines[metric+"_bert"].values, 
    shared_pipelines[metric+"_glove"].values, 
    random=True,
    n_permutations=100000
)
row = {
    "dataset": dataset_name_map[dataset],
    "mean_difference (BERT - GloVE)": observed_diff,
    "p_value": p_val
}
results = pd.DataFrame([row])
latex_table = results.to_latex(
    index=False,
    float_format=simple_format
)
latex_table = process_latex_table(latex_table)

savepath = OUTPUT_DIR / dataset / "tables" / f"{config_str}_glove_vs_bert.tex"
with open(savepath, "w") as f:
    f.write(latex_table)
print(f"Saved to {savepath}")
display(results)

Saved to notebook_output/jc_penney_products/tables/rmse_only_4_replicates_glove_vs_bert.tex


,dataset,mean_difference (BERT - GloVE),p_value
0,jcp,-16.710534,0.00434


Sanity check with a validated t test implementation

In [50]:
# Compare BERT and GloVE with paired t-test - sanity check on permutation test
t_stat, t_pval = stats.ttest_rel(shared_pipelines[metric+"_bert"].values, shared_pipelines[metric+"_glove"].values)

logger.info(f"\nPaired t-test results:")
logger.info(f"T-statistic: {t_stat:.3f}")
logger.info(f"P-value: {t_pval:.3f}")
logger.info(f"Statistically {'different' if t_pval < alpha else 'not different'} at {alpha} significance")


2025-03-11 11:38:13,962 - analysis - INFO - 
Paired t-test results:
2025-03-11 11:38:13,963 - analysis - INFO - T-statistic: -3.252
2025-03-11 11:38:13,965 - analysis - INFO - P-value: 0.003
2025-03-11 11:38:13,966 - analysis - INFO - Statistically different at 0.05 significance


# Do preprocessers matter?

## Preprocessers for traditional NLP

In [51]:
featurisers = ["bow_count", "bow_binary", "tf_idf"]
trad_nlp_results = filtered_results[
    (filtered_results["featuriser"].isin(featurisers))
]
unique_preprocessers = sorted(set(chain.from_iterable(trad_nlp_results["preprocessers"].values)))
uses_preprocessing = trad_nlp_results["preprocessers"].apply(lambda x: x[0] != "empty")
uses_specifically = {}
for preprocesser in unique_preprocessers:
    uses_specifically[preprocesser] = trad_nlp_results["preprocessers"].apply(lambda x: preprocesser in x)

trad_nlp_results["uses_preprocessing"] = uses_preprocessing

fig = px.box(
    trad_nlp_results,
    y=metric,
    color="uses_preprocessing",
    points="outliers"
)
fig.show()


/tmp/ipykernel_56187/1258202587.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [52]:
# Create subplots, one for each preprocesser
fig = make_subplots(cols=len(unique_preprocessers), rows=1, 
                    subplot_titles=unique_preprocessers,
                    shared_yaxes=True,
                    vertical_spacing=0.05)

p_values = []

for i, preprocesser in enumerate(unique_preprocessers, 1):
    _with = trad_nlp_results[uses_specifically[preprocesser]][metric]
    _without = trad_nlp_results[~uses_specifically[preprocesser]][metric]
    # Test normality of each distribution
    _, p_with = stats.normaltest(_with)
    _, p_without = stats.normaltest(_without)
    
    # Perform appropriate statistical test based on normality
    if p_with > 0.05 and p_without > 0.05:
        # Both normal - use t-test
        _, p_val = stats.ttest_ind(_with, _without)
        test_type = "t-test"
    else:
        # At least one non-normal - use Mann-Whitney U test
        _, p_val = stats.mannwhitneyu(_with, _without)
        test_type = "Mann-Whitney U"
    p_values.append(p_val)
    annotation = f"{test_type} p-value: {p_val:.3f}"
    logger.info(f"\n{preprocesser}:")
    logger.info(f"With normality p-value: {p_with:.3f}")
    logger.info(f"Without normality p-value: {p_without:.3f}")
    logger.info(annotation)

    # Add box plot for when preprocesser is used
    fig.add_trace(
        go.Box(
            y=_with,
            name="with",
            boxpoints='outliers',
            marker_color="green"
        ),
        row=1, col=i
    )
    
    # Add box plot for when preprocesser is not used
    fig.add_trace(
        go.Box(
            y=_without,
            name="without",
            boxpoints='outliers',
            marker_color="red"
        ),
        row=1, col=i
    )

    fig.add_hline(y=perf_of_mean, line_dash="dash", line_color="gray")

fig.update_layout(
    width=200*len(unique_preprocessers),
    # showlegend=True,
    title_text="Impact of Different Preprocessors",
    yaxis_title=metric
)


fig.show()

2025-03-11 11:38:39,563 - analysis - INFO - 
empty:
2025-03-11 11:38:39,564 - analysis - INFO - With normality p-value: 0.000
2025-03-11 11:38:39,564 - analysis - INFO - Without normality p-value: 0.000
2025-03-11 11:38:39,566 - analysis - INFO - Mann-Whitney U p-value: 0.086
2025-03-11 11:38:39,602 - analysis - INFO - 
lemmatize:
2025-03-11 11:38:39,606 - analysis - INFO - With normality p-value: 0.000
2025-03-11 11:38:39,608 - analysis - INFO - Without normality p-value: 0.000
2025-03-11 11:38:39,609 - analysis - INFO - Mann-Whitney U p-value: 0.821
2025-03-11 11:38:39,655 - analysis - INFO - 
lowercase:
2025-03-11 11:38:39,656 - analysis - INFO - With normality p-value: 0.000
2025-03-11 11:38:39,656 - analysis - INFO - Without normality p-value: 0.000
2025-03-11 11:38:39,657 - analysis - INFO - Mann-Whitney U p-value: 0.024
2025-03-11 11:38:39,708 - analysis - INFO - 
no_punctuation:
2025-03-11 11:38:39,708 - analysis - INFO - With normality p-value: 0.001
2025-03-11 11:38:39,709 - 

Only takeaway: stemming seems to be universally bad, lemmatizing could be good. Generally this is not meaningful as not stratified by other choices

If something passes then we know at least something even FWER corrected is meaningful to remove/keep

## Specifically for 'modern' dl approaches?

In [54]:
deep_learning_models = ["mlp", "resnet", "deep_linear"]
deep_learning_models = deep_learning_models + [f"{x}_ft" for x in deep_learning_models]
deep_learning_featurizers = ["bert_cls", "bert_mean"]
deep_learning_tokenizers = ["bert"]

deep_learning_results = filtered_results[
    (filtered_results["model"].isin(deep_learning_models)) &
    (filtered_results["featuriser"].isin(deep_learning_featurizers)) &
    (filtered_results["tokeniser"].isin(deep_learning_tokenizers))
]

z = deep_learning_results.sort_values(by=metric).reset_index(drop=True)
logger.info(deep_learning_results.shape)
fig = px.box(deep_learning_results, y=metric, color="model")
fig.add_hline(y=deep_learning_results[metric].mean(), line_dash="dash", line_color="gray")
fig.show()

2025-03-11 11:39:06,198 - analysis - INFO - (116, 24)


In [55]:
logger.info(f"Best deep learners varied over preprocessers:")
best_subset = deep_learning_results.head(5)
display(best_subset[["preprocessers", metric]])

logger.info(f"Worst deep learners varied over preprocessers:")
worst_subset = deep_learning_results.tail(5)
display(worst_subset[["preprocessers", metric]])

2025-03-11 11:39:09,093 - analysis - INFO - Best deep learners varied over preprocessers:


,preprocessers,rmse
1,[no_stopwords_nltk],311.334410
2,[empty],314.764369
3,[empty],316.044175
5,[empty],317.734602
7,[no_stopwords_nltk],319.842292


2025-03-11 11:39:09,104 - analysis - INFO - Worst deep learners varied over preprocessers:


,preprocessers,rmse
561,[empty],404.190459
567,"[no_stopwords_spacy, stem]",405.331844
568,"[no_stopwords_spacy, stem]",405.462961
571,"[lowercase, no_punctuation, stem]",406.013717
593,[stem],422.635551


### Look at with and without preprocesser

In [56]:
no_preprocesser_idx = deep_learning_results["preprocessers"].apply(lambda x: x==["empty"])
no_preprocessers_subset = deep_learning_results[no_preprocesser_idx]
with_preprocessers_subset = deep_learning_results[~no_preprocesser_idx]
no_preprocessers_subset["model_type"] = "w/o Preprocessers"
with_preprocessers_subset["model_type"] = "w/ Preprocessers"
joint = pd.concat([no_preprocessers_subset, with_preprocessers_subset])

color_order = sorted(joint["model"].unique())

fig = px.box(
    joint,
    x="model_type",
    y=metric,
    # color="model",
    points="outliers",
    category_orders={"model": color_order}
)
fig.add_hline(y=perf_of_mean, line_dash="dash", line_color="gray")

fig.write_image(
    OUTPUT_DIR / dataset / "1d_boxplots" / f"{config_str}_deep_learners_w_wout_preprocessers.{image_ext}",
    height=SAVE_HEIGHT,
    width=SAVE_WIDTH,
    scale=2
)

/tmp/ipykernel_56187/4287478704.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_56187/4287478704.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [57]:
p_value, observed_diff = permutation_test(with_preprocessers_subset[metric], no_preprocessers_subset[metric])

logger.info(f"\nPermutation Test Results:")
logger.info(f"Observed difference in means (with - without): {observed_diff:.2f}; n = {len(with_preprocessers_subset)}, m = {len(no_preprocessers_subset)}")
logger.info(f"P-value: {p_value:.4f}")
logger.info(f"Statistically {'different' if p_value < alpha else 'not different'} at {alpha} significance")


2025-03-11 11:39:12,949 - analysis - INFO - 
Permutation Test Results:
2025-03-11 11:39:12,950 - analysis - INFO - Observed difference in means (with - without): 15.25; n = 84, m = 32
2025-03-11 11:39:12,951 - analysis - INFO - P-value: 0.0023
2025-03-11 11:39:12,952 - analysis - INFO - Statistically different at 0.05 significance


Too difficult to draw meaningful conclusions from these results

## 'Safe' vs 'Unsafe' preprocessers

What about specifically preprocessers that do and dont change the token distribution?

In [58]:
# Remind of all preprocessers
all_preproc = set()
for preproc in filtered_results["preprocessers"].values:
    all_preproc.update(preproc)
all_preproc = list(all_preproc)
all_preproc

['lowercase',
 'lemmatize',
 'empty',
 'no_punctuation',
 'no_stopwords_spacy',
 'stem',
 'no_stopwords_nltk']

In [59]:
safe_preproc = ["empty", "no_stopwords_spacy", "lowercase", "no_stopwords_nltk", "no_punctuation", "lemmatize"]
unsafe_preproc = [x for x in all_preproc if x not in safe_preproc]
safe_preproc, unsafe_preproc

(['empty',
  'no_stopwords_spacy',
  'lowercase',
  'no_stopwords_nltk',
  'no_punctuation',
  'lemmatize'],
 ['stem'])

In [60]:

# Condition on unsafe preprocessers
unsafe_results = filtered_results[filtered_results["preprocessers"].apply(
    lambda x: any(y in unsafe_preproc for y in x)
)]

# Condition on safe preprocessers
safe_results = filtered_results[filtered_results["preprocessers"].apply(
    lambda x: all(y in safe_preproc for y in x)
)]

# Create boxplot comparing safe vs unsafe preprocessers
fig = px.box(
    pd.concat([
        safe_results.assign(preproc_type=f"Safe"),
        unsafe_results.assign(preproc_type=f"Unsafe ({unsafe_preproc})")
    ]),
    x="preproc_type",
    y=metric,
    points="outliers",
    color="model",
    title="Performance with Safe vs Unsafe Preprocessors"
)

fig.update_layout(
    xaxis_title="Preprocessor Type",
    yaxis_title=metric.replace("_", " ").title(),
    showlegend=True,
    boxmode='group'
)

In [61]:

fig.write_image(
    OUTPUT_DIR / dataset / "1d_boxplots" / f"{config_str}_safe_vs_unsafe_preprocessers.{image_ext}",
    height=SAVE_HEIGHT,
    width=SAVE_WIDTH,
    scale=2
)

In [62]:
# Run statistical test
p_value, observed_diff = permutation_test(
    safe_results[metric], 
    unsafe_results[metric]
)

logger.info(f"\nPermutation Test Results for Safe vs Unsafe Preprocessors:")
logger.info(f"Observed difference in means (safe - unsafe): {observed_diff:.2f}; n = {len(safe_results)}, m = {len(unsafe_results)}")
logger.info(f"P-value: {p_value:.4f}")
logger.info(f"Statistically {'different' if p_value < alpha else 'not different'} at {alpha} significance")


2025-03-11 11:39:37,496 - analysis - INFO - 
Permutation Test Results for Safe vs Unsafe Preprocessors:
2025-03-11 11:39:37,497 - analysis - INFO - Observed difference in means (safe - unsafe): -9.17; n = 456, m = 164
2025-03-11 11:39:37,498 - analysis - INFO - P-value: 0.0001
2025-03-11 11:39:37,498 - analysis - INFO - Statistically different at 0.05 significance


## Systematically, each preprocesser

In [63]:
# Create plots for each preprocesser showing performance with/without

for preprocesser in all_preproc:
    # Create mask for results that include this preprocesser
    uses_specifically = filtered_results['preprocessers'].apply(lambda x: preprocesser in x)
    
    # Split data
    with_prep = filtered_results[uses_specifically].copy()
    without_prep = filtered_results[~uses_specifically].copy()
    
    # Add labels
    with_prep['preprocesser_usage'] = f'With {preprocesser}'
    without_prep['preprocesser_usage'] = f'Without {preprocesser}'
    
    # Combine data
    plot_data = pd.concat([with_prep, without_prep])
    
    # Create plot
    fig = px.box(
        plot_data,
        x='preprocesser_usage',
        y=metric,
        color='model',
        points='outliers',
        title=f'Performance With vs Without {preprocesser}'
    )
    
    fig.update_layout(
        xaxis_title='Preprocesser Usage',
        yaxis_title=metric.replace('_', ' ').title(),
        showlegend=True,
        boxmode='group'
    )
    
    # Save plot
    plot_dir = OUTPUT_DIR / dataset / '1d_boxplots' / 'preprocessers'
    plot_dir.mkdir(parents=True, exist_ok=True)
    fig.write_image(
        plot_dir / f'{config_str}_with_without_{preprocesser}.{image_ext}',
        height=SAVE_HEIGHT,
        width=SAVE_WIDTH,
        scale=2
    )
    
    # Run statistical test
    p_value, observed_diff = permutation_test(
        with_prep[metric],
        without_prep[metric]
    )
    
    logger.info(f"\nPermutation Test Results for {preprocesser}:")
    logger.info(f"Observed difference in means (with - without): {observed_diff:.2f}; n = {len(with_prep)}, m = {len(without_prep)}")
    logger.info(f"P-value: {p_value:.4f}")
    logger.info(f"Statistically {'different' if p_value < alpha else 'not different'} at {alpha} significance")



2025-03-11 11:39:46,255 - analysis - INFO - 
Permutation Test Results for lowercase:
2025-03-11 11:39:46,256 - analysis - INFO - Observed difference in means (with - without): 4.93; n = 220, m = 400
2025-03-11 11:39:46,256 - analysis - INFO - P-value: 0.0228
2025-03-11 11:39:46,257 - analysis - INFO - Statistically different at 0.05 significance
2025-03-11 11:39:48,736 - analysis - INFO - 
Permutation Test Results for lemmatize:
2025-03-11 11:39:48,736 - analysis - INFO - Observed difference in means (with - without): -1.81; n = 156, m = 464
2025-03-11 11:39:48,737 - analysis - INFO - P-value: 0.4521
2025-03-11 11:39:48,738 - analysis - INFO - Statistically not different at 0.05 significance
2025-03-11 11:39:51,097 - analysis - INFO - 
Permutation Test Results for empty:
2025-03-11 11:39:51,098 - analysis - INFO - Observed difference in means (with - without): -6.46; n = 168, m = 452
2025-03-11 11:39:51,099 - analysis - INFO - P-value: 0.0062
2025-03-11 11:39:51,099 - analysis - INFO -

Conclusion seems to be: don't use stemming; at least when looking at the model partitioned plots.

Stopword removal also works irrespective of pipeline?

# Vector embeddings for GBDTs

In [64]:
vector_embeddings = ["bert_cls", "bert_mean", "glove_mean"]
dl_models = ["mlp", "resnet", "deep_linear"]
dl_models = dl_models + [f"{x}_ft" for x in dl_models]
trad_nlp_with_dl_feat = filtered_results[
    (~filtered_results["model"].isin(dl_models)) &
    (filtered_results["featuriser"].isin(vector_embeddings))
]
trad_nlp_without_dl_feat = filtered_results[
    (~filtered_results["model"].isin(dl_models)) &
    (~filtered_results["featuriser"].isin(vector_embeddings))
]
trad_nlp_with_dl_feat = trad_nlp_with_dl_feat.sort_values(by=metric).reset_index(drop=True)
trad_nlp_without_dl_feat = trad_nlp_without_dl_feat.sort_values(by=metric).reset_index(drop=True)

w_embedding_label = "w/ Vector Embeddings"
wout_embedding_label = "w/o Vector Embeddings"
trad_nlp_with_dl_feat["model_type"] = w_embedding_label
trad_nlp_without_dl_feat["model_type"] = wout_embedding_label
joint = pd.concat([trad_nlp_with_dl_feat, trad_nlp_without_dl_feat])
# Get unique models from both dataframes
all_models = pd.concat([
    trad_nlp_with_dl_feat['model'], 
    trad_nlp_without_dl_feat['model']
]).unique()

# Create color mapping using plotly's default color sequence
from plotly.colors import qualitative
def iqr_threshold(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    return q3 + 1.5 * (q3 - q1) 
model_colors = {
    model: qualitative.Set1[i] for i, model in enumerate(sorted(all_models))
}
model_order = sorted(all_models)

save_name = f"{config_str}_gbdt_w_wout_vec_embeddings"

to_plot = joint
fig = px.box(
    to_plot,
    x="model_type",
    y=metric,
    color="model",
    points="outliers",
    color_discrete_map=model_colors,  # Add this line,
    category_orders={"model": model_order, "model_type": [w_embedding_label, wout_embedding_label]}
)
# fig.update_layout(title="Traditional NLP w/ and w/out vector embeddings")
fig.add_hline(y=perf_of_mean, line_dash="dash", line_color="gray")

fig.write_image(
    OUTPUT_DIR / dataset / "1d_boxplots" / f"{save_name}.{image_ext}",
    height=SAVE_HEIGHT,
    width=SAVE_WIDTH,
    scale=2
)

# Replicate figure without outliers for easier reading

save_name += "_no_outliers"
threshold = iqr_threshold(joint[metric])
k = 10
without_worst_k = joint.sort_values(by=metric).head(len(joint)-k)
to_plot = without_worst_k
fig = px.box(
    to_plot,
    x="model_type",
    y=metric,
    color="model",
    points="outliers",
    color_discrete_map=model_colors,  # Add this line,
    category_orders={"model": model_order, "model_type": [w_embedding_label, wout_embedding_label]}
)
# fig.update_layout(title="Traditional NLP w/ and w/out vector embeddings")
fig.add_hline(y=perf_of_mean, line_dash="dash", line_color="gray")

fig.write_image(
    OUTPUT_DIR / dataset / "1d_boxplots" / f"{save_name}.{image_ext}",
    height=SAVE_HEIGHT,
    width=SAVE_WIDTH,
    scale=2
)
print(f"Saved to {OUTPUT_DIR / dataset / '1d_boxplots' / f'{save_name}.{image_ext}'}")
print(f"Model order: {model_order}")

Saved to notebook_output/jc_penney_products/1d_boxplots/rmse_only_4_replicates_gbdt_w_wout_vec_embeddings_no_outliers.png
Model order: ['catboost', 'xgboost']


In [65]:
# Get scores for each group
with_vec = trad_nlp_with_dl_feat[metric].values
without_vec = trad_nlp_without_dl_feat[metric].values

# Perform permutation test
p_value, diff = permutation_test(without_vec, with_vec)
comparison_df = pd.DataFrame({
    'dataset': [dataset_name_map[dataset]],
    'mean_difference (BoW - Vector)': [diff],
    'p_value': [p_value]
})

latex_table = comparison_df.to_latex(
    index=False,
    float_format=lambda x: f"{x:.3f}"
)
latex_table = process_latex_table(latex_table)
with open(table_dir / f"{config_str}_compare_gbdt_w_wout_vec_embeddings.tex", "w") as f:
    f.write(latex_table)


# Should we choose CLS or Mean?

In [66]:
# Extract RMSE values for CLS and MEAN
cls_rmse = filtered_results[filtered_results['featuriser'] == 'bert_cls'][metric]
mean_rmse = filtered_results[filtered_results['featuriser'] == 'bert_mean'][metric]

# Perform Mann-Whitney U test since we can't assume normality
statistic, p_value = stats.mannwhitneyu(cls_rmse, mean_rmse, alternative='two-sided')

logger.info(f"Mann-Whitney U test results:")
logger.info(f"Statistic: {statistic}")
logger.info(f"p-value: {p_value}")
if p_value < alpha:
    logger.info(f"There is a difference between BERT CLS and BERT MEAN at {alpha} significance")
else:
    logger.info(f"No difference between BERT CLS and BERT MEAN at {alpha} significance")

# Create box plot comparing distributions
fig = go.Figure()

fig.add_trace(go.Box(
    y=cls_rmse,
    name='BERT CLS',
    boxpoints='all',
    jitter=0.3,
    pointpos=-1.8
))

fig.add_trace(go.Box(
    y=mean_rmse,
    name='BERT MEAN', 
    boxpoints='all',
    jitter=0.3,
    pointpos=-1.8
))

fig.update_layout(
    title='RMSE Distribution: BERT CLS vs MEAN',
    yaxis_title='RMSE',
    showlegend=True
)

fig.show()


2025-03-11 11:40:02,884 - analysis - INFO - Mann-Whitney U test results:
2025-03-11 11:40:02,885 - analysis - INFO - Statistic: 6424.0
2025-03-11 11:40:02,886 - analysis - INFO - p-value: 0.0004643343399973647
2025-03-11 11:40:02,887 - analysis - INFO - There is a difference between BERT CLS and BERT MEAN at 0.05 significance


# Can we look at spread associated with choosing a model?

In [67]:
to_analyze = filtered_results.copy()
nlp_steps = ["preprocesser(s)", "tokeniser", "featuriser"]
to_analyze["nlp_hash"] = to_analyze[nlp_steps].apply(lambda x: "_".join(x), axis=1)

## Expected Variance/Standard deviation

In [68]:
from itertools import chain
def expected_agg_over_model(aggregate):
    sub_pipeline_cols = ["preprocesser(s)", "tokeniser", "featuriser"]
    # First remove variance from choosing replicate
    hashable_replicates = filtered_results.copy()
    hashable_replicates = hashable_replicates.groupby(sub_pipeline_cols+["model"]).agg(
        {metric: "mean", "seed": "count"}
    ).reset_index()
    hashable_replicates["hash"] = hashable_replicates[sub_pipeline_cols].apply(lambda x: "_".join(x), axis=1)
    # Find every pipeline that has just the model different at least once
    replicate_idx = hashable_replicates.duplicated(subset="hash", keep=False)
    variance_from_choosing_model = hashable_replicates[replicate_idx].groupby("hash").agg(
        {metric: aggregate}
    )
    result = variance_from_choosing_model.reset_index()
    expected_variance_from_choosing_model = variance_from_choosing_model[metric].mean()
    logger.info(f"Expected {aggregate} from choosing model: {expected_variance_from_choosing_model:.2f}")
    return result, expected_variance_from_choosing_model

In [69]:
def expected_agg_over_step(aggregate, step="model"):
    # Find every pipeline that has a fixed model but varying pipeline
    # We guarantee that each model has duplicates in the dataset
    sub_pipeline_cols = ["preprocesser(s)", "tokeniser", "featuriser"]
    to_analyze = filtered_results.copy()
    to_analyze = to_analyze.groupby(sub_pipeline_cols+["model"]).agg(
        {metric: "mean", "seed": "count"}
    ).reset_index()
    variance_per_model = to_analyze.groupby(step).agg(
        {metric: aggregate}
    )
    result = variance_per_model.reset_index()
    variance_marginalised_over_model = variance_per_model[metric].mean()
    logger.info(f"Expected {aggregate} from choosing everything else marginalised over {step}: {variance_marginalised_over_model:.2f}")
    return result, variance_marginalised_over_model

In [70]:
def expected_agg_over_replicate(aggregate):
    pipeline_cols = ["preprocesser(s)", "tokeniser", "featuriser", "model"]
    hashable_replicates = filtered_results.copy()
    hashable_replicates["hash"] = hashable_replicates[pipeline_cols].apply(lambda x: "_".join(x), axis=1)
    replicate_idx = hashable_replicates.duplicated(subset="hash", keep=False)
    agg_per_replicate = hashable_replicates[replicate_idx].groupby("hash").agg(
        {metric: aggregate}
    ).reset_index()

    result = agg_per_replicate
    agg_marginalised_over_replicate = agg_per_replicate[metric].mean()
    logger.info(f"Expected {aggregate} from choosing replicate: {agg_marginalised_over_replicate:.2f}")
    return result, agg_marginalised_over_replicate


In [71]:
agg = "var"
per_model, overall_model = expected_agg_over_model(agg)
per_upstream, overall_upstream = expected_agg_over_step(agg)
# _ = expected_agg_over_step("std", step="featuriser")
# _ = expected_agg_over_step("std", step="tokeniser")
per_replicate, overall_replicate = expected_agg_over_replicate(agg)
if agg == "var":
    logger.info(f"Overall variance: {filtered_results[metric].var():.2f}")
elif agg == "std":
    logger.info(f"Overall standard deviation: {filtered_results[metric].std():.2f}")

# Create LaTeX table comparing variances
agg_map = {
    "var": "V",
    "std": "S"
}

latex_table = (
    "\\begin{tabular}{lrr}\n"
    "\\toprule\n"
    f"Dataset & $ \\mathbb{{E}}_\\text{{model}}[{agg_map[agg]}]$ & $ \\mathbb{{E}}_\\text{{upstream}}[{agg_map[agg]}]$ \\\\\n"
    "\\midrule\n"
    f"{dataset_name_map[dataset]} & {overall_model:.1e} & {overall_upstream:.1e} \\\\\n"
    "\\bottomrule\n"
    "\\end{tabular}\n"
)
with open(table_dir / f"{config_str}_variance_comparison.tex", "w") as f:
    f.write(latex_table)

print(latex_table)



2025-03-11 11:40:26,630 - analysis - INFO - Expected var from choosing model: 342.91
2025-03-11 11:40:26,639 - analysis - INFO - Expected var from choosing everything else marginalised over model: 369.74
2025-03-11 11:40:26,649 - analysis - INFO - Expected var from choosing replicate: 179.57
2025-03-11 11:40:26,650 - analysis - INFO - Overall variance: 674.24


\begin{tabular}{lrr}
\toprule
Dataset & $ \mathbb{E}_\text{model}[V]$ & $ \mathbb{E}_\text{upstream}[V]$ \\
\midrule
jcp & 3.4e+02 & 3.7e+02 \\
\bottomrule
\end{tabular}



In [72]:
# Create DataFrame comparing variances
variance_df = pd.DataFrame({
    'Dataset': [dataset_name_map[dataset]],
    'var_from_model': [overall_model],
    'var_from_upstream': [overall_upstream], 
    'var_from_replicates': [overall_replicate]
})
print(variance_df.to_latex(
    index=False,
    float_format=lambda x: f"{x:.0e}"
))

\begin{tabular}{lrrr}
\toprule
Dataset & var_from_model & var_from_upstream & var_from_replicates \\
\midrule
jcp & 3e+02 & 4e+02 & 2e+02 \\
\bottomrule
\end{tabular}



In [73]:
print("Per replicate")
display(per_replicate)
print("Per upstream")
display(per_upstream)
print("Per model")
display(per_model)


Per replicate


,hash,rmse
0,"Le,Lo,NP,NSCC_bert_bert_cls_deep_linear",3.032022
1,"Le,Lo,NP,NSCC_bert_bert_mean_catboost",74.457079
2,"Le,Lo,NP,NSCC_stanford_glove_mean_catboost",111.431112
3,"Le,Lo,NP,NSCC_stanford_tf_idf_catboost",390.980812
4,"Le,Lo,NP,NSCC_treebank_bow_binary_deep_linear",153.754821
...,...,...
150,empty_whitespace_glove_mean_deep_linear,1.073295
151,empty_whitespace_glove_mean_mlp,35.716746
152,empty_whitespace_tf_idf_deep_linear,17.878356
153,empty_whitespace_tf_idf_mlp,109.998829


Per upstream


,model,rmse
0,catboost,112.542651
1,deep_linear,752.333735
2,deep_linear_ft,98.390618
3,mlp,1057.236967
4,mlp_ft,198.131577
5,resnet,91.926496
6,resnet_ft,101.523726
7,xgboost,545.815385


Per model


,hash,rmse
0,"Le,Lo,NP,NSCC_treebank_bow_binary",0.211553
1,"Le,Lo,NP,NSN_bert_bert_cls",757.961442
2,"Le,Lo_bert_bert_mean",667.977669
3,"Le,NP,NSCC_bert_bert_mean",200.170961
4,Le_bpe_tf_idf,59.536233
5,"Lo,NP,NSCC,S_stanford_glove_mean",1516.766672
6,"Lo,NP,NSN,S_bert_tf_idf",0.414877
7,"Lo,NP,S_bert_bert_cls",141.625507
8,Lo_bert_bert_cls,224.224810
9,Lo_whitespace_glove_mean,436.153055


# Error improvement?

In [74]:
class CustomReprFunc:

    def __init__(self, f, custom_repr):
        self.f = f
        self.custom_repr = custom_repr

    def __call__(self, *args, **kwargs):
        return self.f(*args, **kwargs)

    def __repr__(self):
        return self.custom_repr(self.f)


def set_repr(custom_repr):
    def set_repr_decorator(f):
        return CustomReprFunc(f, custom_repr)
    return set_repr_decorator

@set_repr(lambda f: f.__name__)
def maximum_improvement(series):
    lowest_metric = series.min()
    improvement = series - lowest_metric
    return improvement.mean() # Mean == expected value

_ = expected_agg_over_model(maximum_improvement)
_ = expected_agg_over_step(maximum_improvement)
_ = expected_agg_over_replicate(maximum_improvement)


2025-03-11 11:40:34,918 - analysis - INFO - Expected maximum_improvement from choosing model: 11.70
2025-03-11 11:40:34,930 - analysis - INFO - Expected maximum_improvement from choosing everything else marginalised over model: 21.00


2025-03-11 11:40:34,977 - analysis - INFO - Expected maximum_improvement from choosing replicate: 11.25
